# Create a Custom Perturbation in a Material

Create a perturbation in a material with a custom function described with [SymPy](https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html) expressions. 

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set perturbation parameters in cell 1.2. (or use default).
1. Set custom perturbation function with SymPy in cell 1.3.
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Notes

1. For more information, see [Introduction](Introduction.ipynb)


## 1. Prepare the Environment

### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("")

### 1.2. Set Generic Perturbation Parameters

In [ ]:
# Set whether to preserve geodesic distance and scale the cell accordingly to match PBC
PRESERVE_GEODESIC_DISTANCE = True

# Set the supercell matrix to apply to original material
SUPERCELL_MATRIX = [[30, 0, 0], [0, 30, 0], [0, 0, 1]]

# Set whether to use Cartesian coordinates for the perturbation function
USE_CARTESIAN_COORDINATES = False

### 1.3. Define Custom Perturbation Function
Provide a [SymPy](https://docs.sympy.org/latest/tutorials/intro-tutorial/intro.html) expression for the perturbation function. The expression should be a function of `x`, `y` and `z` variables.

In [ ]:
import sympy as sp

# Variables for the perturbation function (for SymPy)
variable_names = ["x", "y", "z"]
x, y, z = sp.symbols(variable_names)

# Set the parameters for your perturbation function
AMPLITUDE = 0.05
WAVELENGTH = 1.0
PHASE = 0.0

# Define the perturbation function
custom_sympy_function = AMPLITUDE * sp.sin(2 * sp.pi * x / WAVELENGTH + PHASE) * sp.sin(
    2 * sp.pi * y / WAVELENGTH)

### 1.4. Get input materials
Materials are loaded with `get_materials()`.

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

### 1.5. Create and preview Supercell

In [ ]:
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.supercell import create_supercell

unit_cell = materials[0]
supercell = create_supercell(unit_cell, supercell_matrix=SUPERCELL_MATRIX)
visualize(supercell, repetitions=[1, 1, 1], rotation="0x")

## 2. Create a target material
### 2.1. Set custom perturbation parameters


In [ ]:
from mat3ra.made.tools.build.perturbation import CellMatchingDistancePreservingSlabPerturbationBuilder, \
    PerturbationConfiguration, SlabPerturbationBuilder
from mat3ra.made.tools.utils.perturbation import PerturbationFunctionHolder

custom_perturbation_function = PerturbationFunctionHolder(function=custom_sympy_function,
                                                          variables=variable_names)
configuration_custom = PerturbationConfiguration(
    material=supercell,
    perturbation_function_holder=custom_perturbation_function,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES)

if PRESERVE_GEODESIC_DISTANCE:
    builder = CellMatchingDistancePreservingSlabPerturbationBuilder()
else:
    builder = SlabPerturbationBuilder()

### 2.2. Apply perturbation to the material

In [ ]:
from mat3ra.made.tools.build.perturbation import create_perturbation

material_with_custom_perturbation = create_perturbation(configuration_custom, builder)

### 3. Visualize the Material

In [ ]:
visualize([
    {"material": material_with_custom_perturbation, "title": f"Material with custom perturbation"},
    {"material": material_with_custom_perturbation, "title": f"Material with custom perturbation", "rotation": "-90x"}
])

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(material_with_custom_perturbation)